# TimesFM 2.5 (PyTorch) による Loto データ予測の動作確認

**目的**:
ローカルの PostgreSQL データベース (`dataset.loto_y_ts`) からデータを取得し、`loto` (ロト種別) および `ts_type` (時系列タイプ) ごとに `y` (値) の将来予測を行います。

**環境**:
- Model: TimesFM 2.5 (200M parameters, PyTorch version)
- Data Source: Local PostgreSQL (`dataset.loto_y_ts`)
- Path: `/mnt/e/env/ts/lib_ana/src/model/timesfm/timesfmV2.ipynb`

**前提条件**:
- 必要なライブラリ (`torch`, `pandas`, `numpy`, `sqlalchemy`, `psycopg2-binary`, `timesfm` 等) がインストールされていること。
- Hugging Face のモデル `google/timesfm-2.5-200m-pytorch` へのアクセスが可能であること。

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from sqlalchemy import create_engine
from textwrap import dedent

# timesfm モジュールがパスに通っているか確認してください
# 必要であれば sys.path.append() で調整します
# sys.path.append("../../../") 

import timesfm

# ログ設定の抑制（任意）
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

print(f"Python: {sys.version}")
print(f"Torch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

Python: 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
Torch: 2.4.1+cu121
CUDA Available: True


## データベースからのデータ取得

In [4]:
# データベース接続情報の定義
DB_CONFIG = {
    "user": "loto",
    "password": "z",
    "host": "127.0.0.1",
    "port": "5432",
    "database": "loto"
}

# 接続文字列の作成 (PostgreSQL)
db_url = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# エンジンの作成
engine = create_engine(db_url)

# データ取得クエリ
# loto, ts_type ごとに時系列順 (ds) にデータを取得します
query = """
SELECT
    loto,
    ts_type,
    ds,
    y
FROM
    dataset.loto_y_ts
ORDER BY
    loto,
    ts_type,
    ds ASC
"""

try:
    print("Fetching data from database...")
    df = pd.read_sql(query, engine)
    print(f"Data loaded successfully. Shape: {df.shape}")
    display(df.head())
except Exception as e:
    print(f"Error loading data: {e}")

Fetching data from database...
Data loaded successfully. Shape: (456096, 4)


,loto,ts_type,ds,y
0,bingo5,cumsum,2017-04-05,10.0
1,bingo5,cumsum,2017-04-05,19.0
2,bingo5,cumsum,2017-04-05,13.0
3,bingo5,cumsum,2017-04-05,23.0
4,bingo5,cumsum,2017-04-05,1.0


In [9]:
!pip list

Package                            Version                Build Editable project location
---------------------------------- ---------------------- ----- -------------------------------------------------
absl-py                            2.4.0
accelerate                         1.12.0
accelerator                        2025.11.11
adagio                             0.2.6
ai4ts                              0.0.3
aioboto3                           12.4.0
aiobotocore                        2.12.3
aiohappyeyeballs                   2.6.1
aiohttp                            3.13.3
aiohttp-cors                       0.8.1
aioitertools                       0.13.0
aiosignal                          1.4.0
alabaster                          1.0.0
alembic                            1.18.0
altair                             6.0.0
annotated-doc                      0.0.4
annotated-types                    0.7.0
antlr4-python3-runtime             4.9.3
anyio                              4.12.1
appdi

## データの前処理 (TimesFM入力形式への変換)

In [5]:
# 時系列データをモデル入力用に整形
# TimesFM は List[np.ndarray] 形式の入力を期待します

# ユニークIDの作成（管理用）
df['unique_id'] = df['loto'] + "_" + df['ts_type']

# グルーピングしてリスト化
grouped = df.groupby('unique_id')

input_ts_list = []
metadata_list = []

for name, group in grouped:
    # yカラムをnumpy配列として抽出
    # float32型に変換しておくと無難です
    y_values = group['y'].values.astype(np.float32)
    
    # データが空でないか確認
    if len(y_values) > 0:
        input_ts_list.append(y_values)
        metadata_list.append({
            "unique_id": name,
            "loto": group['loto'].iloc[0],
            "ts_type": group['ts_type'].iloc[0],
            "last_ds": group['ds'].iloc[-1]
        })

print(f"Prepared {len(input_ts_list)} time series for forecasting.")
# サンプルとして最初の系列の長さを表示
if input_ts_list:
    print(f"Sample 0 length: {len(input_ts_list[0])}")

Prepared 36 time series for forecasting.
Sample 0 length: 3648


## モデルのロードとコンパイル

In [6]:
# モデルの初期化 (PyTorch 200M版)
# 初回実行時はHugging Face Hubからモデルをダウンロードします
try:
    print("Loading TimesFM model...")
    model = timesfm.TimesFM_2p5_200M_torch.from_pretrained(
        "google/timesfm-2.5-200m-pytorch",
        torch_compile=True  # 高速化のためTrue推奨 (Linux環境等)
    )
    
    # 予測設定のコンパイル
    # データセットの長さや予測したい期間に合わせてパラメータを調整してください
    PRED_LEN = 12  # 予測期間（ホライズン）
    CONTEXT_LEN = 512 # 入力コンテキスト長（データの長さに応じて調整）

    model.compile(
        timesfm.ForecastConfig(
            max_context=CONTEXT_LEN,
            max_horizon=PRED_LEN,
            normalize_inputs=True, # 入力データの正規化を行う
            use_continuous_quantile_head=True,
            force_flip_invariance=True,
            infer_is_positive=False, # データが負の値を取りうる場合はFalse
            fix_quantile_crossing=True,
        )
    )
    print("Model loaded and compiled successfully.")

except Exception as e:
    print(f"Error loading model: {e}")
    # 代替案: torch_compile=False で再試行
    if "compile" in str(e):
         print("Retrying without torch_compile...")
         model = timesfm.TimesFM_2p5_200M_torch.from_pretrained(
            "google/timesfm-2.5-200m-pytorch",
            torch_compile=False
        )

Loading TimesFM model...


config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

Downloaded.


/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/torch/cuda/__init__.py:230: UserWarning: 
NVIDIA GeForce RTX 5070 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5070 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


Compiling model...
Model loaded and compiled successfully.


## Model Loading

In [8]:
import sys
import os
import torch

# ステップ1で特定したパスをここにハードコードしておくと次回から楽です
# 例: SRC_PATH = "/mnt/e/env/ts/lib_ana/timesfm/src"
# if SRC_PATH not in sys.path: sys.path.insert(0, SRC_PATH)

from timesfm.timesfm_2p5.timesfm_2p5_torch import TimesFM_2p5_200M_torch
import timesfm

print("Loading TimesFM model...")
try:
    # モデルのロード
    model = TimesFM_2p5_200M_torch.from_pretrained(
        "google/timesfm-2.5-200m-pytorch",
        torch_compile=True
    )
    
    # コンパイル設定
    model.compile(
        timesfm.ForecastConfig(
            max_context=512,
            max_horizon=12,
            normalize_inputs=True,
            use_continuous_quantile_head=True,
            force_flip_invariance=True,
            infer_is_positive=False,
            fix_quantile_crossing=True,
        )
    )
    print("Model ready.")
    
except Exception as e:
    print(f"Error: {e}")

Loading TimesFM model...


config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

Downloaded.
Compiling model...
Model ready.


/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/torch/cuda/__init__.py:230: UserWarning: 
NVIDIA GeForce RTX 5070 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5070 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


In [13]:
import os, sys, importlib

print("cwd:", os.getcwd())
print("sys.path[0]:", sys.path[0])

import timesfm
print("imported timesfm from:", timesfm.__file__)


cwd: /mnt/e/env/ts/lib_ana
sys.path[0]: /home/az/miniconda3/envs/ts/lib/python311.zip
imported timesfm from: /home/az/miniconda3/envs/ts/lib/python3.11/site-packages/timesfm/__init__.py


## ファイル作成＆実行（フルパス）

In [10]:
from pathlib import Path

ANALYZER_PATH = Path("/mnt/e/env/ts/lib_ana/src/model/timesfm/_analysis/param_analyzer.py")
ANALYZER_PATH.parent.mkdir(parents=True, exist_ok=True)

# 上のフルコードを、ここにそのまま貼って書き出してもOK。
# 既に作成済みなら不要。
print("Analyzer path:", str(ANALYZER_PATH))


Analyzer path: /mnt/e/env/ts/lib_ana/src/model/timesfm/_analysis/param_analyzer.py


## 1. TimesFM モデルの初期化とロード
事前学習済みのTimesFMモデル (2.5 200M) をロードします。PyTorchバックエンドを使用し、チェックポイントを指定して初期化します。

In [16]:
import torch
from timesfm import TimesFM_2p5_200M_torch

# GPUが利用可能か確認
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# モデルの読み込み
# huggingfaceのリポジトリIDまたはローカルパスを指定
# backend: 'torch' (または 'flax')
tfm = TimesFM_2p5_200M_torch.from_pretrained(
    model_id="google/timesfm-2.5-200m-pytorch",
    device_map=device,
    # 必要に応じてローカルキャッシュやトークンを指定可能
    # cache_dir=None,
    # force_download=False,
    # token=None
)

print("Model loaded successfully.")

Using device: cuda


TypeError: ModelHubMixin.from_pretrained() missing 1 required positional argument: 'pretrained_model_name_or_path'

## 2. 予測設定 (ForecastConfig) の定義
推論時の最大コンテキスト長、最大ホライズン、入力の正規化など、予測に関する詳細なパラメータを設定します。ここでは可能な限りすべての引数を明示します。

In [17]:
from timesfm.configs import ForecastConfig

# 予測設定の作成
forecast_config = ForecastConfig(
    max_context=512,       # モデルに入力する最大過去データ点数 (最大2048まで)
    max_horizon=96,        # 一度に予測する最大ステップ数
    normalize_inputs=True, # 入力を正規化するかどうか (推奨: True)
    per_core_batch_size=32,# バッチサイズ
    
    # 以下は高度な設定 (デフォルト値を明示)
    window_size=None,                  # 特定のウィンドウサイズが必要な場合指定
    use_continuous_quantile_head=False,# 連続分位点ヘッドの使用有無
    force_flip_invariance=False,       # 反転不変性を強制するか (aX+b の処理)
    infer_is_positive=False,           # 出力が非負であることを保証するか
    fix_quantile_crossing=True,        # 分位点の交差を修正するか
    return_backcast=False              # 過去データの再構成(backcast)を返すか
)

print("ForecastConfig configured.")

ForecastConfig configured.


## 3. モデルのコンパイル (Compile)
作成した `forecast_config` をモデルに適用し、推論グラフをコンパイルします。これにより推論が最適化されます。

In [18]:
# モデルのコンパイル
# backend='torch' の場合、torch.compileなどが内部で呼ばれる可能性があります（実装依存）
tfm.compile(
    forecast_config=forecast_config,
    # 追加のkwargsがある場合ここに渡す
)

print("Model compiled.")

NameError: name 'tfm' is not defined

## 4. 時系列予測の実行 (Forecast)
準備した `input_ts_list` (List[np.ndarray]) を使用して予測を実行します。

In [19]:
import numpy as np

# 予測したい期間（ホライズン）を指定
horizon_len = 96 

# 予測の実行
# input_ts_list はユーザーコードで作成済みの List[np.ndarray] を想定
point_forecast, experimental_quantile_forecast = tfm.forecast(
    inputs=input_ts_list,
    horizon=horizon_len
)

# 結果の確認
print(f"Point Forecast Shape: {point_forecast.shape}")
# (Batch_Size, Horizon)

if experimental_quantile_forecast is not None:
    print(f"Quantile Forecast Shape: {experimental_quantile_forecast.shape}")
    # (Batch_Size, Horizon, Quantiles)

NameError: name 'tfm' is not defined

## 5. 外部共変量を使用した予測 (Forecast with Covariates)
`forecast_with_covariates` メソッドを使用して、動的・静的な共変量を加味した予測を行います。引数をすべて使用するため、ダミーの共変量データを作成して渡します。

In [20]:
# --- 共変量データのダミー作成 (引数全使用のため) ---
# 実際のデータがある場合はそれを使用してください

num_series = len(input_ts_list)
# 各系列の入力長を取得
input_lens = [len(x) for x in input_ts_list]

# 1. 動的数値共変量 (Dynamic Numerical): 例 [気温, 湿度]
# train側: 入力系列と同じ長さである必要があります
train_dyn_num = {
    "temp": [np.random.randn(l).tolist() for l in input_lens],
    "humid": [np.random.randn(l).tolist() for l in input_lens]
}
# test側: 予測期間 (horizon_len) と同じ長さである必要があります
test_dyn_num = {
    "temp": [np.random.randn(horizon_len).tolist() for _ in range(num_series)],
    "humid": [np.random.randn(horizon_len).tolist() for _ in range(num_series)]
}

# 2. 動的カテゴリ共変量 (Dynamic Categorical): 例 [曜日, 祝日]
# 注: TimesFM自体は数値入力を期待するため、カテゴリは数値エンコードされているか、
# 内部のxreg_libが処理できる形式である必要があります。ここではリスト形式で渡します。
train_dyn_cat = {
    "dow": [np.random.randint(0, 7, l).tolist() for l in input_lens]
}
test_dyn_cat = {
    "dow": [np.random.randint(0, 7, horizon_len).tolist() for _ in range(num_series)]
}

# 3. 静的数値共変量 (Static Numerical): 例 [店舗面積]
# 各系列につき1つの値
static_num = {
    "area": [np.random.rand() for _ in range(num_series)]
}

# 4. 静的カテゴリ共変量 (Static Categorical): 例 [店舗タイプ]
# 各系列につき1つの値
static_cat = {
    "store_type": [np.random.randint(0, 3) for _ in range(num_series)]
}

# --- 共変量付き予測の実行 ---

cov_point_forecast, cov_xreg_out = tfm.forecast_with_covariates(
    inputs=input_ts_list,
    
    # 動的共変量 (学習期間/コンテキスト)
    dynamic_numerical_covariates={**train_dyn_num}, # train_dynamic_numerical_covariatesに相当
    dynamic_categorical_covariates={**train_dyn_cat},
    
    # 注意: forecast_with_covariatesの実装では、
    # trainとtestの共変量を辞書内で結合して渡すか、内部ロジックに従う必要があります。
    # ライブラリの仕様上、forecast_with_covariatesの引数は以下の通りです:
    # dynamic_numerical_covariates, dynamic_categorical_covariates, 
    # static_numerical_covariates, static_categorical_covariates
    # これらは通常、辞書内で 'train' 部分と 'test' 部分を持つか、
    # ライブラリによってはここですべて渡すと内部で分割、あるいは
    # 呼び出し側で結合して渡すことが求められます。
    # TimesFMのxreg_libの仕様に合わせ、ここでは全ての引数を明示します。
    
    # *実際の実装(timesfm_2p5_base.py)を確認すると引数は以下の通りです*
    # inputs, dynamic_numerical_covariates, dynamic_categorical_covariates,
    # static_numerical_covariates, static_categorical_covariates, ...
    # 辞書の値は Sequence[Sequence[float]] なので、
    # ここでは「コンテキスト + ホライズン」の結合データを渡すのが一般的ですが、
    # 多くの実装ではコンテキスト部分のみ、あるいはAPI仕様によります。
    # ここではAPI定義通り辞書を渡します。
    
    static_numerical_covariates=static_num,
    static_categorical_covariates=static_cat,
    
    # 設定パラメータ
    xreg_mode="xreg + timesfm",  # "xreg + timesfm" (残差予測) or "timesfm + xreg"
    normalize_xreg_target_per_input=True, # 入力ごとにターゲットを正規化するか
    ridge=0.0,                   # リッジ回帰の正則化パラメータ
    max_rows_per_col=0,          # 線形モデル用サブサンプリング (0は無効)
    force_on_cpu=False           # 線形モデル計算をCPUに強制するか
)

print(f"Covariate Forecast Shape: {cov_point_forecast.shape}")

NameError: name 'tfm' is not defined

In [21]:
# Auto-generated call stub (CLE V6)

# NOTE: TODO のところを埋めてください

from timesfm.configs import ForecastConfig

obj = ForecastConfig(
    max_context=0,
    max_horizon=0,
    normalize_inputs=False,
    window_size=0,
    per_core_batch_size=1,
    use_continuous_quantile_head=False,
    force_flip_invariance=True,
    infer_is_positive=True,
    fix_quantile_crossing=False,
    return_backcast=False,
)

In [22]:
obj

ForecastConfig(max_context=0, max_horizon=0, normalize_inputs=False, window_size=0, per_core_batch_size=1, use_continuous_quantile_head=False, force_flip_invariance=True, infer_is_positive=True, fix_quantile_crossing=False, return_backcast=False)